# Inference

To make inferences with the model, first you need to format and place your files in the following directories:

- _database/processed/P1000_adjusted_TPM.csv This file will contain your sample ID-s and CpG-gene beta values.
- _database/processed/response_paper.csv This file contains the sample ID-s and the corresponding age.
- _database/splits/ This directory contains the train-test-validation set for the datasets, and by default only contains the test set of the pan-tissue dataset (even if it is named as train set). 
- create a folder named *extracted* in your main directory if you want to extract the final parameters.

# Import the required packages

In [2]:
import sys
from os.path import join, dirname, realpath
#current_dir = dirname(realpath(__file__))
current_dir = "train"
from preprocessing import pre
import subprocess
sys.path.insert(0, dirname(current_dir))
import os
import imp
import logging
import random
import timeit
import datetime
import numpy as np
import tensorflow as tf
from utils.logs import set_logging, DebugFolder
import yaml
from pipeline.train_validate import TrainValidatePipeline
from pipeline.one_split import OneSplitPipeline
from pipeline.crossvalidation_pipeline import CrossvalidationPipeline
from pipeline.LeaveOneOut_pipeline import LeaveOneOutPipeline
import networkx as nx
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
#from data.data_access import Data
from data.prostate_paper.data_reader import ProstateDataPaper
from copy import deepcopy
import logging

from sklearn import svm, linear_model
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.linear_model import Ridge, ElasticNet, Lasso, SGDClassifier, RidgeClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from analysis.figure_3.data_extraction_utils import get_node_importance, get_link_weights_df_, \
    get_data, get_degrees, adjust_coef_with_graph_degree, get_pathway_names
from model.coef_weights_utils import get_deep_explain_scores
from os import makedirs
from os.path import dirname, realpath, exists
import pickle
from model.model_utils import get_coef_importance
from model import nn
from analysis.figure_3.data_extraction_utils import get_node_importance, get_link_weights_df_, \
    get_data, get_degrees, adjust_coef_with_graph_degree, get_pathway_names
from utils.loading_utils import DataModelLoader
#from config_path import PROSTATE_LOG_PATH, POSTATE_PARAMS_PATH
PROSTATE_LOG_PATH = "_logs/XAI-AGE/RUN1/crossvalidation_average_reg_10_tanh"
POSTATE_PARAMS_PATH = "train/"

Using TensorFlow backend.


# Functions required for the inference

In [ ]:
def transform_prediction(pred_list, age_adult = 20):
    return [(1+age_adult)*np.exp(pred)-1 if pred < 0 else (1+age_adult)*pred+age_adult for pred in pred_list]

def extract_features( x_train, x_test):
        if self.features_params == {}:
            return x_train, x_test
        logging.info('feature extraction ....')

        proc = feature_extraction.get_processor(self.features_params)
        if proc:
            x_train = proc.transform(x_train)
            x_test = proc.transform(x_test)

            if scipy.sparse.issparse(x_train):
                x_train = x_train.todense()
                x_test = x_test.todense()
        return x_train, x_test
def predict( model, x_test, y_test):
        logging.info('predicitng ...')
        y_pred_test = model.predict(x_test)
        if hasattr(model, 'predict_proba'):
            y_pred_test_scores = model.predict_proba(x_test)[:, 1]
        else:
            y_pred_test_scores = y_pred_test

        print 'y_pred_test', y_pred_test.shape, y_pred_test_scores.shape
        return y_pred_test, y_pred_test_scores
    
def preprocess( x_train, x_test):
        logging.info('preprocessing....')
        proc = pre.get_processor(self.pre_params)
        if proc:
            proc.fit(x_train)
            x_train = proc.transform(x_train)
            x_test = proc.transform(x_test)

            if scipy.sparse.issparse(x_train):
                x_train = x_train.todense()
                x_test = x_test.todense()
        return x_train, x_test
    
class Data():
    def __init__(self, id, type, params, test_size=0.3, stratify=True):

        self.test_size = test_size
        self.stratify = stratify
        self.data_type = type
        self.data_params = params
        if self.data_type == 'prostate_paper':
            self.data_reader = ProstateDataPaper(**params)
        else:
            logging.error('unsupported data type')
            raise ValueError('unsupported data type')

    def get_train_validate_test(self):
        return self.data_reader.get_train_validate_test()

    def get_train_test(self):
        x_train, x_validate, x_test, y_train, y_validate, y_test, info_train, info_validate, info_test, columns = self.data_reader.get_train_validate_test()
        # combine training and validation datasets
        x_train = np.concatenate((x_train, x_validate))
        y_train = np.concatenate((y_train, y_validate))
        info_train = list(info_train) + list(info_validate)
        return x_train, x_test, y_train, y_test, info_train, info_test, columns

    def get_data(self):
        x = self.data_reader.x
        y = self.data_reader.y
        info = self.data_reader.info
        columns = self.data_reader.columns
        return x, y, info, columns

    def get_relevant_features(self):
        if hasattr(self.data_reader, 'relevant_features'):
            return self.data_reader.get_relevant_features()
        else:
            return None
        
def get_train_validate_test(self):
        info = self.info
        x = self.x
        y = self.y
        columns = self.columns
        splits_path = join(PROSTATE_DATA_PATH, 'splits')

        training_file = 'training_set_{}.csv'.format(self.training_split)
        training_set = pd.read_csv(join(splits_path, training_file))

        validation_set = pd.read_csv(join(splits_path, 'validation_set.csv'))
        testing_set = pd.read_csv(join(splits_path, 'test_set.csv'))

        info_train = list(set(info).intersection(training_set.id))
        info_validate = list(set(info).intersection(validation_set.id))
        info_test = list(set(info).intersection(testing_set.id))

        ind_train = info.isin(info_train)
        ind_validate = info.isin(info_validate)
        ind_test = info.isin(info_test)

        x_train = x[ind_train]
        x_test = x[ind_test]
        x_validate = x[ind_validate]

        y_train = y[ind_train]
        y_test = y[ind_test]
        y_validate = y[ind_validate]

        info_train = info[ind_train]
        info_test = info[ind_test]
        info_validate = info[ind_validate]

        return x_train, x_validate, x_test, y_train, y_validate, y_test, info_train.copy(), info_validate, info_test.copy(), columns
class DataModelLoader():
    def __init__(self, params_file):
        self.dir_path = os.path.dirname(os.path.realpath(params_file))
        model_parmas, data_parmas = self.load_parmas(params_file)
        data_reader = Data(**data_parmas)
        self.model = None
        x_train, x_validate_, x_test_, y_train, y_validate_, y_test_, info_train, info_validate_, info_test_, cols = data_reader.get_train_validate_test()

        self.x_train = x_train
        self.x_test = np.concatenate([x_validate_, x_test_], axis=0)

        self.y_train = y_train
        self.y_test = np.concatenate([y_validate_, y_test_], axis=0)

        self.info_train = info_train
        self.info_test = list(info_validate_) + list(info_test_)
        self.columns = cols

    def get_data(self):
        return self.x_train, self.x_test, self.y_train, self.y_test, self.info_train, self.info_test, self.columns

    def get_model(self, model_name='P-net_params.yml'):
        # if self.model is None:
        self.model = self.load_model(self.dir_path, model_name)
        return self.model

    def load_model(self, model_dir_, model_name):
        # 1 - load architecture
        params_filename = join(model_dir_, model_name + '_params.yml')
        stream = file(params_filename, 'r')
        params = yaml.load(stream)
        # print params
        # fs_model = model_factory.get_model(params['model_params'][0])
        fs_model = model_factory.get_model(params['model_params'])
        # 2 -compile model and load weights (link weights)
        weights_file = join(model_dir_, 'fs/{}.h5'.format(model_name))
        model = fs_model.load_model(weights_file)
        return model

    def load_parmas(self, params_filename):
        stream = file(params_filename, 'r')
        params = yaml.load(stream, Loader=yaml.UnsafeLoader)
        model_parmas = params['model_params']
        data_parmas = params['data_params']
        return model_parmas, data_parmas



# get a model object from a dictionary
# the params is in the format of {'type': 'model_type', 'params' {}}
# an example is params = {'type': 'svr', 'parmas': {'C': 0.025} }

def construct_model(model_params_dict):
    model_type = model_params_dict['type']
    p = model_params_dict['params']
    # logging.info ('model type: ', str(model_type))
    # logging.info('model paramters: {}'.format(p))

    if model_type == 'svr':
        model = svm.SVR(max_iter=5000, **p)

    if model_type == 'knn':
        model = KNeighborsClassifier(**p)

    if model_type == 'svc':
        model = svm.SVC(max_iter=5000, **p)

    if model_type == 'linear_svc':
        model = LinearSVC(max_iter=5000, **p)

    if model_type == 'multinomial':
        model = MultinomialNB(**p)

    if model_type == 'nearest_centroid':
        model = NearestCentroid(**p)

    if model_type == 'bernoulli':
        model = BernoulliNB(**p)

    if model_type == 'sgd':
        model = SGDClassifier(**p)

    if model_type == 'gaussian_process':
        model = GaussianProcessClassifier(**p)

    if model_type == 'decision_tree':
        model = DecisionTreeClassifier(**p)

    if model_type == 'random_forest':
        model = RandomForestClassifier(**p)

    if model_type == 'adaboost':
        model = AdaBoostClassifier(**p)

    if model_type == 'svr':
        model = svm.SVR(max_iter=5000, **p)
    # elif model_type == 'dt':
    #     # from sklearn.tree import DecisionTreeClassifier
    #     # model = DecisionTreeClassifier(**p)
    #     model = ModelWrapper(model)
    # elif model_type == 'rf':
    #     # from sklearn.ensemble import RandomForestClassifier
    #     model = RandomForestClassifier(**p)
    #     model = ModelWrapper(model)

    if model_type == 'ridge_classifier':
        model = RidgeClassifier(**p)

    elif model_type == 'ridge':
        model = Ridge(**p)


    elif model_type == 'elastic':
        model = ElasticNet(**p)
    elif model_type == 'lasso':
        model = Lasso(**p)
    elif model_type == 'randomforest':
        model = DecisionTreeRegressor(**p)

    elif model_type == 'extratrees':
        from sklearn.ensemble import ExtraTreesClassifier
        model = ExtraTreesClassifier(**p)
        # print model

    elif model_type == 'randomizedLR':
        from sklearn.linear_model import RandomizedLogisticRegression
        model = RandomizedLogisticRegression(**p)

    elif model_type == 'AdaBoostDecisionTree':
        DT_params = params['DT_params']
        model = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(**DT_params), **p)
    elif model_type == 'RandomForestRegressor':
        model = RandomForestRegressor(**p)
    elif model_type == 'ranksvm':
        model = RankSVMKernel()
    elif model_type == 'logistic':
        logging.info('model class {}'.format(model_type))
        model = linear_model.LogisticRegression()

    elif model_type == 'nn':
        model = nn.Model(**p)

    return model


def get_model(params):
    if type(params['params']) == dict:
        model = construct_model(params)
    else:
        model = params['params']
    return model
def train_predict_crossvalidation(self, model_params, X, y, info, cols, model_name):
        logging.info('model_params: {}'.format(model_params))
        n_splits = self.pipeline_params['params']['n_splits']
        skf = StratifiedKFold(n_splits=n_splits, random_state=123, shuffle=True)
        i = 0
        scores = []
        model_list = []
        for train_index, test_index in skf.split(X, y.ravel().astype(int)):
            model = get_model(model_params)
            logging.info('fold # ----------------%d---------' % i)
            x_train, x_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            info_train = pd.DataFrame(index=info[train_index])
            info_test = pd.DataFrame(index=info[test_index])
            x_train, x_test = self.preprocess(x_train, x_test)
            # feature extraction
            logging.info('feature extraction....')
            x_train, x_test = self.extract_features(x_train, x_test)

            model = model.fit(x_train, y_train)

            y_pred_test, y_pred_test_scores = self.predict(model, x_test, y_test)
            score_test = self.evaluate(y_test, y_pred_test, y_pred_test_scores)
            logging.info('model {} -- Test score {}'.format(model_name, score_test))
            self.save_prediction(info_test, y_pred_test, y_pred_test_scores, y_test, i, model_name)

            if hasattr(model, 'save_model'):
                logging.info('saving coef')
                save_model(model, model_name + '_' + str(i), self.directory)

            if self.save_train:
                logging.info('predicting training ...')
                y_pred_train, y_pred_train_scores = self.predict(model, x_train, y_train)
                self.save_prediction(info_train, y_pred_train, y_pred_train_scores, y_train, i, model_name,
                                     training=True)

            scores.append(score_test)

            fs_parmas = deepcopy(model_params)
            if hasattr(fs_parmas, 'id'):
                fs_parmas['id'] = fs_parmas['id'] + '_fold_' + str(i)
            else:
                fs_parmas['id'] = fs_parmas['type'] + '_fold_' + str(i)

            model_list.append((model, fs_parmas))
            i += 1
        self.save_coef(model_list, cols)
        logging.info(scores)
        return scores
def save_gradient_importance(node_weights_, node_weights_samples_dfs, info, samplename):
    for i, k in enumerate(layers[:-1]):
        n = node_weights_[k]
        filename = join(saving_dir, 'gradient_importance_{}_'+samplename+'.csv'.format(i))
        n.to_csv(filename)

    for i, k in enumerate(layers[:-1]):
        n = node_weights_samples_dfs[k]
        if i > 0:
            n['ind'] = info
            n = n.set_index('ind')
            filename = join(saving_dir, 'gradient_importance_detailed_{}_'+samplename+'.csv'.format(i))
            n.to_csv(filename)


def save_link_weights(link_weights_df, layers):
    for i, l in enumerate(layers):
        link = link_weights_df[l]
        filename = join(saving_dir, 'link_weights_{}.csv'.format(i))
        link.to_csv(filename)


def save_activation(layer_outs_dict, feature_names, info):
    for l_name, l_outut in sorted(layer_outs_dict.iteritems()):
        if l_name.startswith('h'):
            print(l_name, l_outut.shape)
            l = int(l_name[1:])
            features = feature_names[l_name]
            layer_output_df = pd.DataFrame(l_outut, index=info, columns=features)
            layer_output_df = layer_output_df.round(decimals=3)
            filename = join(saving_dir, 'activation_{}.csv'.format(l + 1))
            layer_output_df.to_csv(filename)


def save_graph_stats(degrees, fan_outs, fan_ins, layers):
    i = 1

    df = pd.concat([degrees[0], fan_outs[0]], axis=1)
    df.columns = ['degree', 'fan_out']
    df['fan_in'] = 0
    filename = join(saving_dir, 'graph_stats_{}.csv'.format(i))
    df.to_csv(filename)

    for i, (d, fin, fout) in enumerate(zip(degrees[1:], fan_ins, fan_outs[1:])):
        df = pd.concat([d, fin, fout], axis=1)
        df.columns = ['degree', 'fan_in', 'fan_out']
        print(df.head())
        filename = join(saving_dir, 'graph_stats_{}.csv'.format(i + 2))
        df.to_csv(filename)
def save_gradient_importance(node_weights_, node_weights_samples_dfs, info, samplename):
    for i, k in enumerate(layers[:-1]):
        n = node_weights_[k]
        filename = join(saving_dir, 'gradient_importance_'+str(i)+'.csv'.format(i))
        print(filename)
        n.to_csv(filename)

    for i, k in enumerate(layers[:-1]):
        n = node_weights_samples_dfs[k]
        if i > 0:
            n['ind'] = info
            n = n.set_index('ind')
            filename = join(saving_dir, 'gradient_importance_detailed_'+str(i)+'.csv'.format(i))
            n.to_csv(filename)

# Define the type of run (a modified crossvalidation_average_reg_10_tanh by default based on Elmarakeby et al.)

In [ ]:
params_file_list = []


params_file_list.append('./crossvalidation_average_reg_10_tanh')
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

random_seed = 234
random.seed(random_seed)
np.random.seed(random_seed)
tf.random.set_random_seed(random_seed)

timeStamp = '_{0:%b}-{0:%d}_{0:%H}-{0:%M}'.format(datetime.datetime.now())
def elapsed_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs
for params_file in params_file_list:
    log_dir = join(PROSTATE_LOG_PATH, params_file)
    log_dir = log_dir
    set_logging(log_dir)
    params_file = join(POSTATE_PARAMS_PATH, params_file)
    logging.info('random seed %d' % random_seed)
    params_file_full = params_file + '.py'
    print params_file_full
    params = imp.load_source(params_file, params_file_full)

    DebugFolder(log_dir)
    if params.pipeline['type'] == 'one_split':
        pipeline = OneSplitPipeline(task=params.task, data_params=params.data, model_params=params.models,
                                    pre_params=params.pre, feature_params=params.features,
                                    pipeline_params=params.pipeline,
                                    exp_name=log_dir)

    elif params.pipeline['type'] == 'crossvalidation':
        pipeline = CrossvalidationPipeline(task=params.task, data_params=params.data, feature_params=params.features,
                                           model_params=params.models, pre_params=params.pre,
                                           pipeline_params=params.pipeline, exp_name=log_dir)
    elif params.pipeline['type'] == 'Train_Validate':
        pipeline = TrainValidatePipeline(data_params=params.data, model_params=params.models, pre_params=params.pre,
                                         feature_params=params.features, pipeline_params=params.pipeline,
                                         exp_name=log_dir)

    elif params.pipeline['type'] == 'LOOCV':
        pipeline = LeaveOneOutPipeline(task=params.task, data_params=params.data, feature_params=params.features,
                                       model_params=params.models, pre_params=params.pre,
                                       pipeline_params=params.pipeline, exp_name=log_dir)
    start = timeit.default_timer()
    #pipeline.run()

# Load the weights

In [3]:


for data_params in pipeline.data_params:
    data_id = data_params['id']
    # logging
    logging.info('loading data..1..')
    data = Data(**data_params)

    x_train, x_validate_, x_test_, y_train, y_validate_, y_test_, info_train, info_validate_, info_test_, cols = data.get_train_validate_test()

    X = np.concatenate((x_train, x_validate_), axis=0)
    y = np.concatenate((y_train, y_validate_), axis=0)
    info = np.concatenate((info_train, info_validate_), axis=0)

    # get model
    logging.info('fitting model ...')
    
    
   

    for model_param in pipeline.model_params:
        if 'id' in model_param:
            model_name = model_param['id']
        else:
            model_name = model_param['type']

        #set_random_seeds(random_seed=20080808)
        model_name = model_name + '_' + data_id
        m_param = deepcopy(model_param)
        m_param['id'] = model_name
        logging.info('fitting model ...')
# load model
# load model and data --------------------
#model_dir = join(base_dir, model_name)
from model import model_factory
model_file = 'XAI_AGE_ALL'
params_file = join("_logs/XAI-AGE/RUN1/crossvalidation_average_reg_10_tanh/", model_file + '_params.yml')
print(params_file)
loader = DataModelLoader(params_file)
nn_model = loader.get_model(model_file)
feature_names = nn_model.feature_names



setting logs
random seed 234
train/./crossvalidation_average_reg_10_tanh.py
{'params': {'save_train': True, 'n_splits': 2}, 'type': 'crossvalidation'}
loading data..1..
loading gene_expression
loading data from _database/processed/P1000_adjusted_TPM.csv,


train/./crossvalidation_average_reg_10_tanh.py:1: RuntimeWarning: Parent module 'train/' not found while handling absolute import
  from model.builders.prostate_models import build_pnet2


(1619, 15591)
loading response from response_paper.csv
loaded data 1619 samples, 15591 variables, 1619 responses 
15591
[set(['GPX4_cg17812013', 'MTMR6_cg15071391', 'CTNNBIP1_cg27308387', 'B3GAT2_cg18403396', 'TNFSF13_cg17267493', 'CHCHD7_cg11321895', 'TULP3_cg22016818', 'CLCN2_cg22613010', 'TMOD4_cg02301754', 'ZNF540_cg17333479', 'MAPK11_cg00164898', 'TRIP12_cg05670275', 'SNX2_cg17425616', 'ARFGAP1_cg21944757', 'CUX1_cg02862153', 'EPCAM_cg07059875', 'KDM5A_cg11225935', 'ST5_cg16176301', 'UTP3_cg17849194', 'SYN3_cg05288803', 'NUDT21_cg25974870', 'TRMT6_cg27367554', 'GATA4_cg01546563', 'LOXL3_cg26117023', 'ATP4A_cg06123346', 'CD2BP2_cg05116896', 'CLPS_cg25107791', 'PTPRO_cg10646402', 'SCO1_cg12478185', 'KMT2A_cg13656360', 'MRPL3_cg04322344', 'SLC39A5_cg03343942', 'UQCRC1_cg16129988', 'PGAM5_cg17903544', 'GSTM4_cg11903880', 'CHEK2_cg22585269', 'SYVN1_cg02441831', 'ACSF2_cg06818777', 'RGL2_cg02990567', 'TTC37_cg00849368', 'ORC1_cg10586599', 'PANK3_cg16421589', 'PDPN_cg18877506', 'NAT2_cg1

/v/xgrp0j/.conda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:378: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.


[[0.01391966 0.8528583  0.02803011 ... 0.97520066 0.05008657 0.02512847]
 [0.04222079 0.89695727 0.06959295 ... 0.96564265 0.11005506 0.07648358]
 [0.03731636 0.80432243 0.04802043 ... 0.93545836 0.21877556 0.05705917]
 ...
 [0.1269233  0.8973955  0.06760386 ... 0.9477908  0.2996123  0.07339883]
 [0.09950238 0.8916971  0.07512545 ... 0.9525703  0.288933   0.0700205 ]
 [0.1009163  0.9445524  0.07075141 ... 0.9653937  0.328225   0.07788645]]
[[62.68219178]
 [58.64931507]
 [63.1890411 ]
 ...
 [79.        ]
 [92.        ]
 [71.        ]]
After combining, loaded data 1619 samples, 15591 variables, 1619 responses 
fitting model ...
fitting model ...
_logs/XAI-AGE/RUN1/crossvalidation_average_reg_10_tanh/XAI_AGE_ALL_params.yml
loading gene_expression
loading data from _database/processed/P1000_adjusted_TPM.csv,
loading from memory cached_data
(1619, 15591)
loading from memory cached_data
loaded data 1619 samples, 15591 variables, 1619 responses 
15591
[set(['GPX4_cg17812013', 'MTMR6_cg1507139

...

hello
<dictionary-valueiterator object at 0x7fb001462310>
layer # 0
pathways 1387
genes 170805
filtered_map (15591, 0)
filtered_map (15591, 0)
filtered_map (15591, 0)
layer 0 , # of edges  45659.0
layer # 1
pathways 1066
genes 1399
filtered_map (1387, 0)
filtered_map (1387, 0)
filtered_map (1387, 0)
layer 1 , # of edges  1396.0
layer # 2
pathways 447
genes 1068
filtered_map (1066, 0)
filtered_map (1066, 0)
filtered_map (1066, 0)
layer 2 , # of edges  1070.0
layer # 3
pathways 147
genes 448
filtered_map (447, 0)
filtered_map (447, 0)
filtered_map (447, 0)
layer 3 , # of edges  447.0
layer # 4
pathways 26
genes 147
filtered_map (147, 0)
filtered_map (147, 0)
filtered_map (147, 0)
layer 4 , # of edges  148.0
layer # 5
pathways 1
genes 26
filtered_map (26, 0)
filtered_map (26, 0)
filtered_map (26, 0)
layer 5 , # of edges  26.0
original dropout [0.5, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
dropout [1, 2, 3, 4, 5] [0.5, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1] [0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001]
n_

model/builders/builders_utils.py:228: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="linear", kernel_regularizer=<keras.reg..., name="o_linear2")`
  W_regularizer=reg_l(w_reg_outcome))(outcome)
model/builders/builders_utils.py:228: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="linear", kernel_regularizer=<keras.reg..., name="o_linear3")`
  W_regularizer=reg_l(w_reg_outcome))(outcome)
model/builders/builders_utils.py:228: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="linear", kernel_regularizer=<keras.reg..., name="o_linear4")`
  W_regularizer=reg_l(w_reg_outcome))(outcome)
model/builders/builders_utils.py:228: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="linear", kernel_regularizer=<keras.reg..., name="o_linear5")`
  W_regularizer=reg_l(w_reg_outcome))(outcome)
model/builders/builders_utils.py:228: UserWarning: Update your `Dense` call to the Keras 2 A

layer 4, dropout  0.1 w_reg 0.001
Compiling...
loss_weights [2, 7, 20, 54, 148, 400]


model/builders/prostate_models.py:171: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=[<tf.Tenso...)`
  model = Model(input=[ins], output=outcome)


done compiling
  - 0 inputs (None, 15591)
  - 1 h0 (None, 15591)
  - 2 dropout_0 (None, 15591)
  - 3 h1 (None, 1387)
  - 4 dropout_1 (None, 1387)
  - 5 h2 (None, 1066)
  - 6 dropout_2 (None, 1066)
  - 7 h3 (None, 447)
  - 8 dropout_3 (None, 447)
  - 9 h4 (None, 147)
  - 10 dropout_4 (None, 147)
  - 11 h5 (None, 26)
  - 12 o_linear1 (None, 1)
  - 13 o_linear2 (None, 1)
  - 14 o_linear3 (None, 1)
  - 15 o_linear4 (None, 1)
  - 16 o_linear5 (None, 1)
  - 17 o_linear6 (None, 1)
  - 18 o1 (None, 1)
  - 19 o2 (None, 1)
  - 20 o3 (None, 1)
  - 21 o4 (None, 1)
  - 22 o5 (None, 1)
  - 23 o6 (None, 1)
[<keras.engine.input_layer.InputLayer object at 0x7fb0490d5d90>, <model.layers_custom.Diagonal object at 0x7fb000676e10>, <keras.layers.core.Dropout object at 0x7faff8cc4c50>, <model.layers_custom.SparseTF object at 0x7faf5e062f10>, <keras.layers.core.Dropout object at 0x7fb0014df150>, <model.layers_custom.SparseTF object at 0x7fb0014df110>, <keras.layers.core.Dropout object at 0x7faf7e4af210>, <mo

# Make predictions

In [5]:
y_pred_test, y_pred_test_scores = predict(nn_model, X, y)
predictions = pd.DataFrame({'ID': info,'Label':np.concatenate(y).ravel(),'Prediction':np.concatenate(y_pred_test).ravel()})
predictions = predictions.drop_duplicates(subset=['ID'])


Here you can save your predictions to a csv file:

In [8]:
predictions.to_csv("", index = False)

# Extracting features

In [ ]:
PROSTATE_LOG_PATH = "_logs/XAI-AGE/RUN1/crossvalidation_average_reg_10_tanh"

In [9]:

#current_dir = dirname(dirname(realpath(__file__)))
current_dir = os.path.dirname(os.path.abspath("__file__"))

sys.path.insert(0, dirname(current_dir))

#from config_path import *


samplename = ''
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

current_dir = os.path.dirname(os.path.abspath("__file__"))

saving_dir = join(current_dir, 'extracted')

if not exists(saving_dir):
    makedirs(saving_dir)


base_dir = join(PROSTATE_LOG_PATH, '')
model_name = ''

importance_type = ['deepexplain_deeplift']
target = 'o6'
use_data = 'Test'  # {'All', 'Train', 'Test'}
dropAR = False

layers = ['inputs', 'h0', 'h1', 'h2', 'h3', 'h4', 'h5', 'o_linear6']



# load model and data --------------------
model_dir = join(base_dir, model_name)
model_file = 'XAI_AGE_ALL'
params_file = join(model_dir, model_file + '_params.yml')
print(params_file)
loader = DataModelLoader(params_file)
nn_model = loader.get_model(model_file)
feature_names = nn_model.feature_names
X, Y, info = get_data(loader, use_data, dropAR)
response = pd.DataFrame(Y, index=info, columns=['response'])
print(response.head())
filename = join(saving_dir, 'response.csv')
response.to_csv(filename)
#
print('saving gradeint importance')
# #gradeint importance --------------------
#node_weights_, node_weights_samples_dfs = get_node_importance(nn_model, X, Y, importance_type[0], target)



    
model = nn_model.model



x_train = X
y_train = Y
info_train = info
importance_type_new=importance_type[0]

#ret = get_coef_importance(model, x_train, y_train, target=target, feature_importance=importance_type_new, detailed=True)
method = importance_type_new.split('_')[1]
ret = get_deep_explain_scores(model, x_train, y_train, target, method_name=method, detailed=True)
print(type(ret))
if type(ret) is tuple:
    coef, coef_detailed = ret
    print('coef_detailed', len(coef_detailed))

else:
    coef = ret
    # empty
    coef_detailed = [c.T for c in coef]

node_weights_dfs = {}
node_weights_samples_dfs = {}

for i, k in enumerate(nn_model.feature_names.keys()):
    name = nn_model.feature_names[k]
    w = coef[k]
    w_samples = coef_detailed[k]
    features = get_pathway_names(name)
    df = pd.DataFrame(abs(w.ravel()), index=name, columns=['coef'])
    layer = pd.DataFrame(index=name)
    layer['layer'] = i
    # node_weights_dfs.append(df)
    node_weights_dfs[k] = df
    # layers.append(layer)
    df_samples = pd.DataFrame(w_samples, columns=features)
    # node_weights_samples_dfs.append(df_samples)
    node_weights_samples_dfs[k] = (df_samples)


node_weights_=node_weights_dfs
node_weights_samples_dfs = node_weights_samples_dfs

save_gradient_importance(node_weights_, node_weights_samples_dfs, info_train, str(samplename))
#
print('saving link weights')
# # link weights --------------------
link_weights_df = get_link_weights_df_(nn_model.model, feature_names, layers)
save_link_weights(link_weights_df, layers[1:])
#
print('saving activation')
# # activation --------------------
#layer_outs_dict = nn_model.get_layer_outputs(X)
#save_activation(layer_outs_dict, feature_names, info_train)
#
print('saving graph stats')
# # graph stats --------------------
stats = get_degrees(link_weights_df, layers[1:])
import numpy as np
keys = np.sort(stats.keys())
for k in keys:
    filename = join(saving_dir, 'graph_stats_{}.csv'.format(k))
    stats[k].to_csv(filename)
# save_graph_stats(degrees,fan_outs, fan_ins)
#
print('adjust weights with graph stats')

# # graph stats --------------------
degrees = []
for k in keys:
    degrees.append(stats[k].degree.to_frame(name='coef_graph'))

node_importance = adjust_coef_with_graph_degree(node_weights_, stats, layers[1:-1], saving_dir)

with open('extracted_data.pkl', 'w') as f:  # Python 3: open(..., 'wb')
        pickle.dump([keys, stats, node_weights_,layers,node_importance], f)

filename = join(saving_dir, 'node_importance_graph_adjusted_test.csv')
node_importance.to_csv(filename)


Your predictions will be saved to the **extracted** folder.